## Test of cbpro library

In [11]:
import pandas as pd
from datetime import date
import cbpro
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [12]:
a = np.array([1,2,3])
np.reshape(a, (-1,1))
a.shape

(3,)

In [13]:
a

array([1, 2, 3])

In [14]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [15]:
public_client = cbpro.PublicClient()

BTC_USDT1 = public_client.get_product_historic_rates('BTC-USDT', '2023-01-01', '2023-06-01', 86400)
ETH_USDT1 = public_client.get_product_historic_rates('ETH-USDT', '2023-01-01', '2023-06-01', 86400)
BTC_USDT2 = public_client.get_product_historic_rates('BTC-USDT', '2022-07-01', '2022-12-31', 86400)
ETH_USDT2 = public_client.get_product_historic_rates('ETH-USDT', '2022-07-01', '2022-12-31', 86400)
BTC_USDT3 = public_client.get_product_historic_rates('BTC-USDT', '2022-01-01', '2022-06-01', 86400)
ETH_USDT3 = public_client.get_product_historic_rates('ETH-USDT', '2022-01-01', '2022-06-01', 86400)
# XRP_USDT = public_client.get_product_historic_rates('XRP-USDT', '2023-01-01', '2023-06-01', 86400)

In [16]:
BTC_USDT = BTC_USDT1 + BTC_USDT2 + BTC_USDT3
ETH_USDT = ETH_USDT1 + ETH_USDT2 + ETH_USDT3

In [17]:
df_hist_data= pd.DataFrame(BTC_USDT)
df_hist_data.columns = [ 'date', 'BTC_low', 'BTC_high', 'BTC_open', 'BTC_close', 'BTC_volume' ]
df_hist_data['date']= df_hist_data['date'].apply(lambda x: date.fromtimestamp(x))
coin_res = [ETH_USDT]
coin_names = ['ETH']
for coin, name in zip (coin_res, coin_names):
    tmp = pd.DataFrame(coin)
    tmp.columns = [ 'date', 'low', 'high', 'open', 'close', 'volume' ]
    df_hist_data[f'{name}_close'] = tmp['close']
df_hist_data.tail()

,date,BTC_low,BTC_high,BTC_open,BTC_close,BTC_volume,ETH_close
483,2022-01-04,42475.00,47069.69,45828.31,43457.96,1038.790522,3538.61
484,2022-01-03,45550.00,47527.94,46446.93,45825.84,440.490606,3784.25
485,2022-01-02,45700.00,47565.16,47270.43,46437.18,387.863115,3764.03
486,2022-01-01,46644.55,47987.85,47718.04,47289.79,367.660559,3827.39
487,2021-12-31,46219.13,47958.65,46221.22,47728.93,313.215971,3766.72


In [18]:
scaler  = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(np.array(df_hist_data['BTC_close'].values).reshape(-1,1) )

In [42]:
prediction_days = 60
x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x,0])
x_train, y_train = (np.array(x_train), np.array(y_train))
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [43]:
model = Sequential()
model.add(LSTM(units = 50, activation='tanh', return_sequences=True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

2024-02-14 12:32:37.566816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-02-14 12:32:37.569226: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-02-14 12:32:37.571044: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [44]:
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25


2024-02-14 12:32:39.006864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-02-14 12:32:39.009706: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-02-14 12:32:39.011947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

14/14 [==============================] - 12s 346ms/step - loss: 0.0597 - accuracy: 0.0047
Epoch 2/25
14/14 [==============================] - 5s 354ms/step - loss: 0.0164 - accuracy: 0.0047
Epoch 3/25
14/14 [==============================] - 3s 201ms/step - loss: 0.0115 - accuracy: 0.0047
Epoch 4/25
14/14 [==============================] - 5s 337ms/step - loss: 0.0106 - accuracy: 0.0047
Epoch 5/25
14/14 [==============================] - 3s 236ms/step - loss: 0.0097 - accuracy: 0.0047
Epoch 6/25
14/14 [==============================] - 5s 337ms/step - loss: 0.0087 - accuracy: 0.0047
Epoch 7/25
14/14 [==============================] - 5s 345ms/step - loss: 0.0096 - accuracy: 0.0047
Epoch 8/25
14/14 [==============================] - 4s 245ms/step - loss: 0.0087 - accuracy: 0.0047
Epoch 9/25
14/14 [==============================] - 4s 274ms/step - loss: 0.0083 - accuracy: 0.0047
Epoch 10/25
14/14 [==============================] - 5s 354ms/step - loss: 0.0098 - accuracy: 0.0047
Epoch 11/